## Import libary and data

In [1]:
import os
import math
import torch
import pandas as pd

from transformers import GPT2Tokenizer, GPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer
from huggingface_hub import notebook_login
from datasets import Dataset
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
! gdown --id 1hCMS4n0h9bb7jfeQhotZ-Nv_fnYXC9YT

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1hCMS4n0h9bb7jfeQhotZ-Nv_fnYXC9YT
To: /kaggle/working/poem_dataset_final.zip
100%|██████████████████████████████████████| 37.6k/37.6k [00:00<00:00, 64.3MB/s]


In [3]:
!unzip /kaggle/working/poem_dataset_final.zip

Archive:  /kaggle/working/poem_dataset_final.zip
  inflating: poem_final.csv          


In [4]:
DATASET_PATH = "/kaggle/working/poem_final.csv"
df = pd.read_csv(DATASET_PATH)
df

,Unnamed: 0,title,content,source,url
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
1,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...
2,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...
3,3,"“Đến, nhiều nơi để đến”","Đến, nhiều nơi để đến\nVề, trở lại với mình\nC...","Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
4,4,“Đừng bao giờ dại dột”,Đừng bao giờ dại dột\nĐem chuyện riêng của mìn...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...
...,...,...,...,...,...
185,95,Ám ảnh sông xưa,"Ôi, con sóng chết khô,\nvật vờ trong bùn quánh...",NaN,https://www.thivien.net/%C4%90%E1%BB%97-Qu%E1%...
186,96,Áng dương không biết sầu,Áng dương không biết sầu\nNằm mãi ở trên cao\n...,"Nguồn: Lâu Văn Mua, Tôi bay vào mắt em (thơ), ...",https://www.thivien.net/L%C3%A2u-V%C4%83n-Mua/...
187,97,Anh,Cây bút gẫy trong tay\nCặn mực khô đáy lọ\nÁnh...,19-7-1973\n\n[Thông tin 2 nguồn tham khảo đã đ...,https://www.thivien.net/Xu%C3%A2n-Qu%E1%BB%B3n...
188,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...


## Setup dataframe to datasets for hunggingface 

### Split part poem

In [5]:
def split_content(content : str):
    sample = []

    poem_parts = content.split("\n\n")
    for part in poem_parts:
        sentences = part.split("\n")
        if len(sentences) == 4:
            sample.append(sentences)

    return sample

df['part_content'] = df['content'].apply(lambda x : split_content(x))
df['part_content']

0      [[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhi...
1      [[Chiều vừa xốp trên tay, Chợt nghe thoáng ong...
2      [[Dưới giàn hoa thiên lý, Một mình anh đang ng...
3      [[Đến, nhiều nơi để đến, Về, trở lại với mình,...
4      [[Đừng bao giờ dại dột, Đem chuyện riêng của m...
                             ...                        
185    [[Ôi, con sóng chết khô,, vật vờ trong bùn quá...
186    [[Áng dương không biết sầu, Nằm mãi ở trên cao...
187                                                   []
188    [[Không có anh để già, Làm sao em được trẻ, Mu...
189    [[Buồn, thì buồn thiệt đó, Em à, có biết không...
Name: part_content, Length: 190, dtype: object

### Explode content to new dataframe

In [6]:
df_exploded = df.explode("part_content")
df_exploded.reset_index(inplace=True, drop=True)
df_exploded

,Unnamed: 0,title,content,source,url,part_content
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...,"[Cái làm ta hạnh phúc, Thực ra cũng chẳng nhiề..."
1,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...,"[Rồi thêm chút công việc, Cho ta làm hàng ngày..."
2,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...,"[Chiều vừa xốp trên tay, Chợt nghe thoáng ong ..."
3,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...,"[Ớt đỏ sao cứ đỏ, Táo chín cho thật vàng, Em đ..."
4,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...,"[Dưới giàn hoa thiên lý, Một mình anh đang ngồ..."
...,...,...,...,...,...,...
481,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"[Đất hiến dâng tuổi trẻ, Trời buông thả tuổi g..."
482,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"[Buồn, thì buồn thiệt đó, Em à, có biết không,..."
483,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"[Vâng, phiến buồn còn đó, Em nào biết phải khô..."
484,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"[Ừ, nỗi buồn vậy đó, Em đừng vội bâng khuâng, ..."


### Clear null

In [7]:
print("Tổng số hàng null", df_exploded['part_content'].isnull().sum())
df_exploded.dropna(inplace=True, subset="part_content")
print("Tổng số hàng null", df_exploded['part_content'].isnull().sum())

Tổng số hàng null 45
Tổng số hàng null 0


### Change list to string

In [8]:
df_exploded["part_content"] = df_exploded["part_content"].apply(lambda x: "\n".join(x))
df_exploded

,Unnamed: 0,title,content,source,url,part_content
0,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...
1,0,“Cái làm ta hạnh phúc”,Cái làm ta hạnh phúc\nThực ra cũng chẳng nhiều...,"Nguồn: Châm ngôn mới (thơ), Thái Bá Tân, NXB L...",https://www.thivien.net/Th%C3%A1i-B%C3%A1-T%C3...,Rồi thêm chút công việc\nCho ta làm hàng ngày\...
2,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...
3,1,“Chiều vừa xốp trên tay”,Chiều vừa xốp trên tay\nChợt nghe thoáng ong b...,"Nguồn: Lâm Huy Nhuận, Chiều có thật (thơ), NXB...",https://www.thivien.net/L%C3%A2m-Huy-Nhu%E1%BA...,Ớt đỏ sao cứ đỏ\nTáo chín cho thật vàng\nEm đẹ...
4,2,“Dưới giàn hoa thiên lý...”,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...,"Nguồn: Nguyễn Nhật Ánh, Mắt biếc, NXB Trẻ, 2004",https://www.thivien.net/Nguy%E1%BB%85n-Nh%E1%B...,Dưới giàn hoa thiên lý\nMột mình anh đang ngồi...
...,...,...,...,...,...,...
481,98,Anh biết,Không có anh để già\nLàm sao em được trẻ\nMuốn...,NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,Đất hiến dâng tuổi trẻ\nTrời buông thả tuổi gi...
482,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n..."
483,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"Vâng, phiến buồn còn đó\nEm nào biết phải khôn..."
484,99,Anh bốn mùa lập đông,"Buồn, thì buồn thiệt đó\nEm à, có biết không\n...",NaN,https://www.thivien.net/Nguy%E1%BB%85n-Minh-D%...,"Ừ, nỗi buồn vậy đó\nEm đừng vội bâng khuâng\nV..."


### Dataloader

In [9]:
TEST_SIZE = 0.1
poem_dataset = Dataset.from_pandas(df_exploded)
poem_dataset = poem_dataset.train_test_split(test_size=TEST_SIZE)
poem_dataset

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', 'part_content', '__index_level_0__'],
        num_rows: 396
    })
    test: Dataset({
        features: ['Unnamed: 0', 'title', 'content', 'source', 'url', 'part_content', '__index_level_0__'],
        num_rows: 45
    })
})

## Preprocessing

In [10]:
tokenizer = GPT2Tokenizer.from_pretrained('danghuy1999/gpt2-viwiki')
model = GPT2LMHeadModel.from_pretrained('danghuy1999/gpt2-viwiki').to(device)

vocab.json:   0%|          | 0.00/773k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/431k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/916 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [11]:
tokenizer.pad_token = tokenizer.eos_token
MAX_LENGTH = 100

def preprocess_function(row):

    encode = tokenizer(
        row["part_content"],
        max_length=MAX_LENGTH,
        padding="max_length",
        truncation=True
    )
    return encode

In [12]:
tokenized_poem_dataset = poem_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=poem_dataset["train"].column_names,
)

Map:   0%|          | 0/396 [00:00<?, ? examples/s]

Map:   0%|          | 0/45 [00:00<?, ? examples/s]

## Training model

### Data colator

In [13]:
data_collator = DataCollatorForLanguageModeling(tokenizer = tokenizer, mlm = False)

In [14]:
import os 
os.environ["WANDB_DISABLED"] = "true"
training_args = TrainingArguments(
    output_dir="/kaggle/working/gpt2_viet_poem_generation",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=20,
    weight_decay=0.01,
    logging_strategy="epoch",
    save_total_limit=1,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_poem_dataset["train"],
    eval_dataset=tokenized_poem_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-14-51e8c35f12bb>:14: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss
25,7.260400
50,6.628400
75,6.345200
100,6.135800
125,5.965800
150,5.795200
175,5.660900
200,5.521100
225,5.407200
250,5.314000


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=500, training_loss=5.476361419677734, metrics={'train_runtime': 243.9497, 'train_samples_per_second': 32.466, 'train_steps_per_second': 2.05, 'total_flos': 404186112000000.0, 'train_loss': 5.476361419677734, 'epoch': 20.0})

## Inferences

In [15]:
prompt = "Học học nữa học mãi\n"

inputs = tokenizer(prompt, return_tensors="pt").input_ids.to(device)
outputs = model.generate(
    inputs,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    temperature=0.8,
    repetition_penalty=1.2
)
results = tokenizer.batch_decode(outputs, skip_special_tokens=True)
results = results[0]
print(results)
for line in results.split("\n"):
    print(line)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Học học nữa học mãi
trong đời mình đời còn lại
Chỉ, những gì đã uống
như anh tài trong làng hội
Những câu chuyện đẹp như là một phù du
Không cần có cả hai ai?
Một ngày lễ lửa sinh ra!
Gây khắp
Học học nữa học mãi
trong đời mình đời còn lại
Chỉ, những gì đã uống
như anh tài trong làng hội
Những câu chuyện đẹp như là một phù du
Không cần có cả hai ai?
Một ngày lễ lửa sinh ra!
Gây khắp
